In [11]:
import os
import pandas as pd
from janome.tokenizer import Tokenizer
from collections import Counter
import PyPDF2
import pandas as pd
import re


target_lang = "en"
DEEPL_API_KEY = input()
# 使用例
directory = "."  # 現在のディレクトリを指定
pdf_files = get_pdf_paths(directory)
pdf_files



def get_pdf_paths(directory):
    # PDFファイルのパスリストを格納するためのリスト
    pdf_paths = []

    # 指定したディレクトリを走査
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                # フルパスを取得してリストに追加
                pdf_paths.append(os.path.join(root, file))

    return pdf_paths



['.\\pdf\\pdf.pdf',
 '.\\pdf\\Python基礎統計1_03.pdf',
 '.\\pdf\\Python基礎統計1_04.pdf',
 '.\\pdf\\Python基礎統計1_05.pdf',
 '.\\pdf\\Python基礎統計1_06.pdf',
 '.\\pdf\\Python基礎統計1_07.pdf',
 '.\\pdf\\Python基礎統計1_08.pdf',
 '.\\pdf\\Python基礎統計1_09.pdf',
 '.\\pdf\\Python基礎統計1_10.pdf',
 '.\\pdf\\Python基礎統計1_11.pdf',
 '.\\pdf\\Python基礎統計1_12.pdf',
 '.\\pdf\\Python基礎統計1_13.pdf',
 '.\\pdf\\可視化の技術2_03.pdf']

In [12]:


# ストップワードのリスト
stop_words = set([
    "する", "ある", "いる", "こと", "これ", "それ", "あれ", "ため", "もの",
    "ここ", "さん", "そう", "これら", "それら", "あれら", "及び", "および", "において", "さらに"
])

# 英単語を検出するための正規表現パターン
english_word_pattern = re.compile(r'[a-zA-Z]+')

# PDFからテキストを抽出する関数
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# 前処理を行う関数
def preprocess_text(text):
    text = re.split('底本：', text)[0]
    text = re.sub('一', '', text, 1)
    text = re.sub('_', '', text, 1)
    text = re.sub('《.+?》', '', text)
    text = re.sub('［＃.+?］', '', text)
    text = re.sub('\n\n', '\n', text)
    text = re.sub(r'\d+', '', text)  # 数字の削除
    text = re.sub(r'[^\w\s]', '', text)  # 記号の削除
    text = text.replace("\n","").replace(" ","")
    return text

# 形態素解析を行い、ストップワードおよび英単語を除いた単語を集計する関数
def analyze_text(text):
    tokenizer = Tokenizer()
    words = []
    for token in tokenizer.tokenize(text):
        base_form = token.base_form
        part_of_speech = token.part_of_speech.split(',')[0]
        reading = token.reading
        if base_form not in stop_words and not english_word_pattern.match(base_form) and part_of_speech != "記号":
            words.append((base_form, part_of_speech, reading))
    return words

# PDFのパスを指定
pdf_path = "pdf/pdf.pdf"

# PDFからテキストを抽出
text = extract_text_from_pdf(pdf_path)

# 前処理を実施
text = preprocess_text(text)
print(text)
# 形態素解析を行い、単語を抽出
word_pos_list = analyze_text(text)

# 単語の出現回数を集計
word_freq = Counter([word for word, pos, reading in word_pos_list])

# データフレームを作成
df = pd.DataFrame(word_pos_list, columns=["単語", "品詞", "ふりがな"])
df["出現回数"] = df["単語"].map(word_freq)

# 重複行を削除し、出現回数でソート
df = df.drop_duplicates(subset=["単語"]).sort_values(by="出現回数", ascending=False).reset_index(drop=True)

# 結果を表示
df


Python基礎統計Python基礎統計第回中心を表す基本特性値の計算記述統計記述統計学とは私たちは興味関心のある対象を観察したり測定したりすることで対象に関するデータを得ることができる集めたデータの多くは数字情報として収集され数のまとまりとして扱われるしかしこの数字のまとまりをただ漠然と眺めているだけではそこから新しい気づきやアイデアビジネスチャンスを得ることは難しいそこで統計学の手法を用いてデータの特徴や性質を調べたりその結果をグラフに示したりすることによってデータへの理解が深まり対象への理解を深めることができる統計学は大まかに記述統計学推測統計学ベイズ統計学に分類されるこのうち記述統計学は収集した目の前にあるデータについて整理要約することを試みる学問分野であるデータから平均や分散相関係数といった指標を計算しデータそのものの特徴や傾向を把握することが目的である方データはその対象の全てを反映しているとは限らない例えばアンケート調査では日本人全体とか在校生全体といった対象の中から時間や予算といったコストの範囲内で人数を絞って調査をするこのとき日本人全体とか在校生全体のことを母集団と呼び母集団の中から選ばれ実際に調査した対象のことを標本と呼ぶ標本を対象に集めたデータから分かるのは標本の性質であるが本来知りたいのは母集団の性質であるこの場合標本の情報から母集団の性質を推定するという操作が必要になりこれを行うのが推測統計学やベイズ統計学である質的変数と量的変数記述統計学において変数とは計測の対象となったもののことを指す例えばスポーツテストを実施して運動能力を測定したときm走のタイムやボールの飛距離握力の数値などが変数になる変数は計測対象の性質に応じて質的変数と量的変数のつに分けることができる質的変数分類や種類を区別するラベルとしてのデータ例段階評価とても良い良い普通悪いとても悪い血液型A型B型O型AB型性別男性女性Python基礎統計量的変数数値自体に意味があり足し算や引き算ができるデータテストの点数気温質的変数のなかでも男女買う買わないのように種類の値しか扱わないとで表現できるデータのことを値変数やバイナリデータとも呼ぶ尺度水準質的変数は名義尺度と順序尺度量的変数は間隔尺度と比例尺度に分類できるこれらつの尺度名義尺度順序尺度間隔尺度比例尺度のことを尺度水準と呼ぶ一般的に

,単語,品詞,ふりがな,出現回数
0,の,助詞,ノ,188
1,を,助詞,ヲ,139
2,値,名詞,チ,93
3,に,助詞,ニ,82
4,データ,名詞,データ,75
...,...,...,...,...
482,概,名詞,ガイ,1
483,況最頻,名詞,*,1
484,操作,名詞,ソウサ,1
485,推定,名詞,スイテイ,1


In [14]:



def translate_text(text, target_lang):
    url = "https://api-free.deepl.com/v2/translate"
    params = {
        "auth_key": DEEPL_API_KEY,
        "text": text,
        "target_lang": target_lang
    }
    response = requests.post(url, data=params)
    if response.status_code == 200:
        return response.json()["translations"][0]["text"]
    else:
        raise Exception(f"Error in translation: {response.status_code}, {response.text}")




In [15]:

def translate_df(df, target_lang):
    # Check if the input is a pandas DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame.")
    
    # Check if the DataFrame contains the "単語" column
    if "単語" not in df.columns:
        raise ValueError('DataFrame must contain a "単語" column.')
    
    # If the DataFrame already contains the "翻訳後" column, return the DataFrame as is
    if "翻訳後" in df.columns:
        return df
    
    # Convert the column to a list of words
    words = df["単語"].tolist()
    result = '\n'.join(words)
    # Translate the text
    try:
        res = translate_text(result, target_lang)
    except Exception as e:
        raise RuntimeError(f"Error during translation: {e}")
    
    # Split the translated text back into a list
    translated_list = res.strip().split('\n')
    
    # Add the translated list to the DataFrame
    df["翻訳後"] = translated_list
    return df


res_df = translate_df(df,target_lang)


の
を
値
に
データ
が
は
_
計算
統計
と
だ
で
平均
て
た
変数
や
尺度
中央
れる
型
できる
求める
場合
も
的
とき
用いる
処理
配列
ない
個数
代表
から
学
なる
量
基礎
ば
その
合計
対象
扱う
数
よう
として
総
呼ぶ
列
という
記述
リスト
番号
パッケージ
による
偶数
使用
つ
数値
取り出す
情報
確認
次
算出
意味
関数
真ん中
関係
多い
フレーム
質
量的
例えば
性質
表す
得点
奇数
といった
中
行う
メソッド
分類
サイズ
よい
特徴
う
例
最も
得る
サンプル
説明
連続
取る
られる
大小
必要
仙台
標本
分かる
たい
インデックス
頻る
分布
最
抽出
など
ファイル
たり
比例
まとまり
順序
図
名義
チーム
母集団
離散
用意
この
調査
コード
全体
指標
分析
したがって
テスト
方法
水準
年
数字
読み込む
ソート
か
公式
プログラム
基本
最初
使う
裾
扱える
推測
間隔
作成
だけ
一般
測定
間
イメージ
入力
式
山形
まで
モンテディオ
文
差
掛け算
に対して
条件
順位
行
全
さ
長い
係数
代入
考える
検索
一方
標準
演習
比率
ぬ
可能
中間
昇順
出現
利用
名
上
部分
テーブル
指定
外部
機能
リーグ
表
性
読み込み
ちょうど
右
実務
左右
外れる
大切
影響
割る
産物
足す
様々
ひと
シュート
沿う
チームベガルタ
集める
気温
とか
在校生
へ
でも
区別
について
結果
そこ
計測
点数
アンケート
ベイズ
足し算
に関する
とても
内
良い
悪い
収集
理解
日本人
関心
買う
全て
把握
中心
距離
方
持つ
性別
表現
種類
分ける
保存
繰り返す
入れる
割り算
によって
やすい
難しい
示す
調べる
かたち
書き直す
どの
再
手法
グラフ
人
他
見返す
見る
活用
いく
上げる
準備
個々
相
データセット
分散
行列
分野
正確
学問
試みる
思う
形式
要約
おく
整理
角
括弧
前
正しい
目
うち
大まか
直接
まずは
深める
深まる
数える
最後
順番
ビジネスチャンス
イン
書き込む
スライス
観察
被
フリー
キック
コーナーキック
反則
警告
退場
ベガルタ
興味
たち
鹿島
アントラーズベガルタ

In [16]:
res_df


,単語,品詞,ふりがな,出現回数,翻訳後
0,の,助詞,ノ,188,"(at sentence-end, falling tone) indicates a co..."
1,を,助詞,ヲ,139,the
2,値,名詞,チ,93,value
3,に,助詞,ニ,82,to
4,データ,名詞,データ,75,Data
...,...,...,...,...,...
482,概,名詞,ガイ,1,Approximate
483,況最頻,名詞,*,1,Status Most Frequent
484,操作,名詞,ソウサ,1,Operation
485,推定,名詞,スイテイ,1,Estimation
